<a href="https://colab.research.google.com/github/NroyAnalyst/Nishant/blob/main/movie_recommendation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  Load dataset from GitHub RAW link
url = "https://raw.githubusercontent.com/yinghaoz1/tmdb-movie-dataset-analysis/master/tmdb-movies.csv"
df = pd.read_csv(url)

# Pick correct columns (title + text to analyze)
title_col = 'title' if 'title' in df.columns else 'original_title'
text_col = 'overview' if 'overview' in df.columns else None

# If no 'overview', combine other metadata into text
if text_col is None:
    parts = [c for c in ['tagline','keywords','genres','cast','director'] if c in df.columns]
    df['__text__'] = df[parts].fillna('').astype(str).agg(' '.join, axis=1)
    text_col = '__text__'

# Drop missing overviews
df = df.dropna(subset=[text_col]).reset_index(drop=True)

#  Convert movie descriptions into vectors (TF-IDF)
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(df[text_col])

#  Compute similarity between all movies
cosine_sim = cosine_similarity(tfidf)

#  Create a mapping: movie title → row index
indices = pd.Series(df.index, index=df[title_col].str.lower()).drop_duplicates()

#  Define recommendation function
def recommend_movies(title, n=5):
    key = title.lower()
    if key not in indices.index:
        # Suggest similar matches if title not found
        matches = df[df[title_col].str.contains(title, case=False, na=False)][title_col].head(5).tolist()
        return ["Movie not found. Did you mean:"] + matches if matches else ["Movie not found. Check spelling."]
    idx = indices[key]
    sims = cosine_sim[idx]
    top_idx = sims.argsort()[::-1][1:n+1]  # skip itself
    return df[title_col].iloc[top_idx].tolist()

#  Test the system
print(" Movies similar to 'Avatar':", recommend_movies("Avatar"))
print(" Movies similar to 'The Dark Knight':", recommend_movies("The Dark Knight"))
print(" Movies similar to 'Interstellar':", recommend_movies("Interstellar"))


 Movies similar to 'Avatar': ['The American', 'Apollo 18', 'The Matrix', 'Hellraiser: Bloodline', 'Europa Report']
 Movies similar to 'The Dark Knight': ['The Dark Knight Rises', 'Batman Returns', 'Batman: Under the Red Hood', 'Batman Forever', 'Batman: The Dark Knight Returns, Part 2']
 Movies similar to 'Interstellar': ['Suburban Commando', 'Stargate', 'Buzz Lightyear of Star Command: The Adventure Begins', 'Starship Troopers', 'The Green Inferno']
